# 📌 Uygulama – 2: “cv::Mat → torch::Tensor + Tek Resimde Inference”

Bu uygulamada şunları öğreneceksin:

* OpenCV ile bir resmi yüklemek

* cv::Mat → torch::Tensor dönüşümü

* Tensor’u modele verip forward almak

* Çıktıyı konsola yazdırmak

* Görselleri okurken kanal sıralaması (BGR → RGB)

* Normalize ve boyut dönüştürme

* Tensor shape kontrolleri

Hepsini adım adım, Jupyter hücresi formatında, soru + çözüm şeklinde yapacağız.

---

## 🚀 Uygulama – 2 / Adım 1
“OpenCV ekle, resmi oku, cv::Mat ekranda göster.”

Aşağıdaki kodu Jupyter Markdown hücresine direkt atıp çalıştıracak şekilde yazıyorum.

### 📌 Adım 1 – OpenCV ile resim okuma ve gösterme

Bu adımda amacım OpenCV’yi projeye bağlamak ve basit bir şekilde bir görüntüyü okuyup ekranda göstermek.
Bu, daha sonra LibTorch ile birlikte kullanacağım temel OpenCV akışının ilk parçasıdır.

### Görev:
1. OpenCV’yi CMake’e ekle
2. Bir resim oku (`test.jpg`)
3. Ekrana göster (`cv::imshow`)
4. Pencerenin kapanmaması için `cv::waitKey(0)` ekle

Aşağıdaki kod, tamamen minimal bir OpenCV testidir.
```cpp
// SORU:
//  - Bir resmi OpenCV ile nasıl okurum ve ekranda nasıl gösteririm?

#include <iostream>
#include <torch/script.h>
#include <opencv2/opencv.hpp>

int main() {
    try {
        // 1) Resim yolu
        cv::Mat img = cv::imread("test.jpg");

        if (img.empty()) {
            std::cerr << "[HATA] test.jpg yüklenemedi." << std::endl;
            return -1;
        }

        std::cout << "[OK] Resim başarıyla yüklendi. Boyut: "
                  << img.cols << "x" << img.rows << std::endl;

        // 2) Ekranda göster
        cv::imshow("Goruntu", img);
        cv::waitKey(0);
    }
    catch (const std::exception& e) {
        std::cerr << "[EXCEPTION] " << e.what() << std::endl;
        return -1;
    }

    return 0;
}
```


---

### 🔧 `CMakeLists.txt` Güncellemesi

```cmake
find_package(OpenCV REQUIRED)
include_directories(${OpenCV_INCLUDE_DIRS})
target_link_libraries(model_test "${OpenCV_LIBS}" "${TORCH_LIBRARIES}")
```

### 📸 Notlar

* test.jpg dosyasını build/Debug/ klasörüne koyarsam direkt çalışır.

>Alternatif: resmin tam yolunu verebilirim.

Örneğin:
```cpp
cv::Mat img = cv::imread("C:/Users/.../Desktop/test.jpg");
```


* Bu adım tamamlandığında görüntü doğru şekilde ekranda açılmalıdır.
* Bu başarıyla çalışıyorsa bir sonraki adıma geçebilirim:


---

# 🔥 Şimdi senden istediğim

Bu adımı uygulayalım:

1. OpenCV’yi CMakeLists’e ekle  

2. `main.cpp`’yi yukarıdaki gibi düzenle 

3. Bir `test.jpg` koy  

4. `cmake --build .`  

5. `model_test.exe` çalıştır

Ve:

**Resim ekranda açıldı mı?**

----

### Gerekli kodlar aşağıdaki gibi

## CMakeList.txt
```bash
cmake_minimum_required(VERSION 3.20 FATAL_ERROR)
project(libtorch_opencv_test)

# LibTorch yolu
set(CMAKE_PREFIX_PATH "C:/libtorch")

# OpenCV CMake config klasörü (sende burası)
set(OpenCV_DIR "C:/opencv/build/x64/vc16/lib")

# OpenCV ve Torch'u bul
find_package(OpenCV REQUIRED)
find_package(Torch REQUIRED)

# Derlenecek dosya
add_executable(model_test main.cpp)

# OpenCV include klasörleri
include_directories(${OpenCV_INCLUDE_DIRS})

# Kütüphaneleri bağla
target_link_libraries(model_test 
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)

# C++ standardı
set_property(TARGET model_test PROPERTY CXX_STANDARD 17)

# LibTorch DLL'lerini exe klasörüne kopyala
add_custom_command(TARGET model_test POST_BUILD
    COMMAND ${CMAKE_COMMAND} -E copy_directory
    "C:/libtorch/bin"
    $<TARGET_FILE_DIR:model_test>
)


```

### 📌 Build oluşturma komutları (tam adımlar)

Projenin kökünde:
```bash
mkdir build
cd build
cmake ..
cmake --build .
```


Hepsi bu kadar.

----

----

# Gelin bir de benim yorumum üzerinden bu işlemlere bakalım:

---

---

# 📁 Proje Yapısı, CMakeLists ve DLL Bağlantıları – Detaylı Açıklama

Bu bölümde LibTorch + OpenCV kullandığım C++ projesinin:

- Klasör yapısını  
- CMakeLists.txt içindeki kritik satırları  
- DLL (LibTorch + OpenCV) bağlantı mantığını  

adım adım, sade bir şekilde anlatıyorum.

---

## 🧱 1) Genel Klasör Yapısı

Proje tarafında temel dizin yapım şu şekilde:

```text
C:/
 ├─ libtorch/                     # LibTorch CPU paketi (bin/include/lib/share)
 ├─ opencv/
 │   └─ build/
 │       ├─ x64/
 │       │   └─ vc16/
 │       │        ├─ bin/         # OpenCV DLL dosyaları (opencv_world4120d.dll vs.)
 │       │        └─ lib/         # OpenCV .lib dosyaları + OpenCVConfig.cmake
 │       └─ ...
 └─ Users/
     └─ .../Masaüstü/C ++/Bölüm - 11 ( LibTorch )/
         └─ Uygulama - 2/
             ├─ CMakeLists.txt
             ├─ main.cpp
             └─ build/
                 ├─ Debug/
                 │   ├─ model_test.exe
                 │   ├─ (LibTorch DLL'leri)
                 │   └─ (OpenCV DLL'leri)
                 └─ ...
```
#### Özetle:

* C:/libtorch → LibTorch’la ilgili tüm include/lib/bin dosyaları burada.

* C:/opencv/build/x64/vc16 → OpenCV’nin CMake config, .lib ve .dll dosyaları burada.

* Uygulama - 2 → Asıl C++ projem (CMakeLists + main.cpp + build klasörü).


---

## 📝 2) CMakeLists.txt Dosyası – Tam Yapı

Projede kullandığım CMakeLists.txt:

```bash
cmake_minimum_required(VERSION 3.20 FATAL_ERROR)
project(libtorch_opencv_test)

# LibTorch CPU yolu
set(CMAKE_PREFIX_PATH "C:/libtorch")

# OpenCV CMake config klasörü (OpenCVConfig.cmake'in bulunduğu yer)
set(OpenCV_DIR "C:/opencv/build/x64/vc16/lib")

# OpenCV ve Torch'u bul
find_package(OpenCV REQUIRED)
find_package(Torch REQUIRED)

# Derlenecek dosya
add_executable(model_test main.cpp)

# OpenCV include klasörleri
include_directories(${OpenCV_INCLUDE_DIRS})

# Kütüphaneleri bağla
target_link_libraries(model_test 
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)

# C++ standardı
set_property(TARGET model_test PROPERTY CXX_STANDARD 17)

# --- DLL kopyalama (LibTorch + OpenCV) ---
add_custom_command(TARGET model_test POST_BUILD
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/libtorch/bin"
        $<TARGET_FILE_DIR:model_test>
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/opencv/build/x64/vc16/bin"
        $<TARGET_FILE_DIR:model_test>
)
```

- Şimdi satır satır ne yaptığını anlatıyorum.
---

## 🔍 3) LibTorch ve OpenCV Yol Tanımları
```bash
set(CMAKE_PREFIX_PATH "C:/libtorch")
set(OpenCV_DIR "C:/opencv/build/x64/vc16/lib")
```

### CMAKE_PREFIX_PATH

**CMake’e “TorchConfig.cmake dosyasını şu kökün altında ara” diyorum.**

**CMake, "C:/libtorch/share/cmake/Torch/TorchConfig.cmake" dosyasını buradan buluyor.**

>Sonuç: find_package(Torch REQUIRED) çalışır hale geliyor.

### OpenCV_DIR

**OpenCV’nin CMake config dosyası olan OpenCVConfig.cmake şu klasörde:**
>C:/opencv/build/x64/vc16/lib/OpenCVConfig.cmake

OpenCV_DIR değişkenine bu klasörü veriyorum.

* Sonuç: find_package(OpenCV REQUIRED) OpenCV’yi başarıyla buluyor.


---

## 🔎 4) find_package Çağrıları

```cpp
find_package(OpenCV REQUIRED)
find_package(Torch REQUIRED)
```


Bu iki satır:

>OpenCV’nin include, lib, dll bilgilerini CMake’e yüklüyor.

> LibTorch’un include, lib, link bilgilerini CMake’e yüklüyor.

Sonrasında:
```bash
${OpenCV_INCLUDE_DIRS}

${OpenCV_LIBS}

${TORCH_LIBRARIES}
```

* gibi değişkenler dolmuş oluyor.
---
## 🧩 5) Executable ve Include Ayarları
```bash
add_executable(model_test main.cpp)
include_directories(${OpenCV_INCLUDE_DIRS})
```


>add_executable → main.cpp’ten model_test.exe oluştur.

>include_directories → OpenCV’nin header dosyalarını (opencv2/opencv.hpp) projemde görülebilir yap.
---
## 🔗 6) Kütüphaneleri Linklemek
```bash
target_link_libraries(model_test 
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)
```


Bu satırda:

* model_test isimli exe, link aşamasında LibTorch ve OpenCV kütüphaneleriyle bağlanıyor.

* TORCH_LIBRARIES → torch.lib, c10.lib, vs. gibi PyTorch’un .lib dosyaları.

* OpenCV_LIBS → opencv_world4120d.lib gibi OpenCV’nin .lib dosyaları.

Bu adım başarılıysa:

* main.cpp içinde hem torch:: hem de cv:: fonksiyonlarını kullanabilirim.

* Link zamanı hataları çözülmüş olur.

---
## 🧠 7) C++ Standardı
```bash
set_property(TARGET model_test PROPERTY CXX_STANDARD 17)
```


* Projenin C++ standardını C++17 olarak kilitliyorum.

* LibTorch ve modern C++ kodları için güvenli bir seviye.
---
# 🧬 8) DLL Kopyalama Mantığı (LibTorch + OpenCV)
```cmake
add_custom_command(TARGET model_test POST_BUILD
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/libtorch/bin"
        $<TARGET_FILE_DIR:model_test>
    COMMAND ${CMAKE_COMMAND} -E copy_directory
        "C:/opencv/build/x64/vc16/bin"
        $<TARGET_FILE_DIR:model_test>
)
```

### Burası Windows ortamında en kritik kısım.

#### Neden gerekli?

Derleme anında:

* Link aşamasında .lib dosyaları kullanılır

* Çalışma zamanında .dll dosyaları gerekir

LibTorch ve OpenCV’nin .dll dosyaları:

**LibTorch için: C:/libtorch/bin**

**OpenCV için: C:/opencv/build/x64/vc16/bin**

Exe’nin olduğu klasörde (build/Debug) bu dll’ler yoksa şu hataları alırım:

* c10.dll bulunamadı

* torch_cpu.dll bulunamadı

* opencv_world4120d.dll bulunamadı

Bu custom command:

>Build bittikten sonra (POST_BUILD)

>C:/libtorch/bin içindeki tüm DLL’leri exe’nin yanına kopyalar

>C:/opencv/build/x64/vc16/bin içindeki tüm DLL’leri de exe’nin yanına kopyalar

### Yani şu yapıyı elde ederim:
```bash
Uygulama - 2/
  build/
    Debug/
      model_test.exe
      c10.dll
      torch_cpu.dll
      opencv_world4120d.dll
      ...
```
Sonuç:

* DLL hatası ortadan kalkar

* Exe, hem LibTorch’u hem OpenCV’yi sorunsuz yükler

---
## 🧾 9) Build Süreci – Baştan Sona Akış

Proje kökünde (CMakeLists.txt ile aynı klasör):
```bash
mkdir build
cd build
cmake ..
```


* CMake, Torch ve OpenCV’yi bulur.

* VS projelerini (.vcxproj) üretir.

### Build:
```bash
cmake --build .
```

* model_test.exe derlenir.

* Link aşaması geçer.

* POST_BUILD adımı çalışır.

* DLL’ler build/Debug klasörüne kopyalanır.

### Çalıştırma:
```bash
cd Debug
model_test.exe
```


* OpenCV ile resim açma

LibTorch ile modeli çalıştırma gibi işlemler yapılır.

----

## 🎯 10) Özet

Bu yapı sayesinde:

* LibTorch ve OpenCV, CMake ile doğru şekilde bulunuyor

* Derleme aşamasında .lib dosyaları ile linkleniyor

* Çalışma zamanında .dll dosyaları otomatik olarak exe yanına kopyalanıyor

* find_package, CMAKE_PREFIX_PATH, OpenCV_DIR ve POST_BUILD mantıklarını net şekilde kullanmış oluyorum

**Böylece LibTorch + OpenCV tabanlı C++ projeler için profesyonel bir temel iskelet kurulmuş oluyor.**
---

---